In [7]:
from psycopg2 import connect
import configparser
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
import folium
from IPython.display import HTML, display
import datetime
import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from folium import plugins
from folium.plugins import HeatMap
import shapely
from shapely.geometry import Point
import fiona
import os
import seaborn as sns
from matplotlib import rcParams
from matplotlib.collections import QuadMesh
from matplotlib.text import Text
import scipy
import matplotlib.gridspec as gridspec
from scipy import stats
import math
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, PercentFormatter)
import matplotlib.font_manager as font_manager
import matplotlib.patches as patches


from pathlib import Path
import configparser
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('config.cfg'))) #Creates a path to your db.cfg file
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)


In [8]:
query = '''

SELECT route_stop, on_street, cross_street, sum(boarding) AS boarding, sum(alighting) as alighting, avg(departure_load) as avg_load FROM ttc.ridership_raw a
LEFT JOIN ttc_stop_card1 b USING (route_stop, stop_card, route, dir)
WHERE a.route = 100
AND a.dir = 'N'
AND b.rting_num = 0
GROUP BY route_stop, cross_street, on_street

'''

stops_100 = pandasql.read_sql(query, con)

In [9]:
stops_100

,route_stop,on_street,cross_street,boarding,alighting,avg_load
0,1,BROADVIEW,STATION,3977.0,1.0,20.080808
1,3,BROADVIEW,AT 845,68.0,8.0,20.383838
2,4,BROADVIEW,BROWNING,11.0,65.0,20.111111
3,5,BROADVIEW,MORTIMER,59.0,165.0,19.575758
4,6,BROADVIEW,BATER,24.0,91.0,19.237374
5,7,BROADVIEW,COSBURN,35.0,164.0,18.585859
6,8,BROADVIEW,TORRENS,6.0,87.0,18.176768
7,9,BROADVIEW,OCONNOR,5.0,78.0,17.808081
8,10,OCONNOR,HOPEDALE,11.0,46.0,17.631313
9,11,PAPE,OCONNOR,23.0,63.0,17.429293


In [10]:
query = '''

WITH time_convert AS (
SELECT route_stop, on_street, cross_street, boarding , alighting , departure_load , 
CASE WHEN tm > 2400 THEN
to_char(tm - 2400, 'FM99909:99')::TIME WITHOUT TIME ZONE
ELSE to_char(tm, 'FM99909:99')::TIME WITHOUT TIME ZONE
END AS tms
 FROM ttc.ridership_raw a
LEFT JOIN ttc_stop_card1 b USING (route_stop, stop_card, route, dir)
WHERE route_stop = 21
AND route = 100
AND dir = 'N'

)

SELECT on_street, cross_street, round(avg(boarding),1) AS boarding, round(avg(alighting),1) as alighting, round(avg(departure_load),1) as avg_load, 
extract(hour fROM tms) as hr
FROM time_convert

GROUP BY route_stop, on_street, cross_street, hr
ORDER BY hr

'''

gateway_donmills = pandasql.read_sql(query, con)
gateway_donmills

,on_street,cross_street,boarding,alighting,avg_load,hr
0,GATEWAY S,DON MILLS,0.0,1.7,12.5,0.0
1,GATEWAY S,DON MILLS,0.0,0.3,5.3,1.0
2,GATEWAY S,DON MILLS,1.0,0.0,5.0,5.0
3,GATEWAY S,DON MILLS,0.0,0.4,7.4,6.0
4,GATEWAY S,DON MILLS,0.2,1.4,6.9,7.0
5,GATEWAY S,DON MILLS,0.7,9.1,12.6,8.0
6,GATEWAY S,DON MILLS,0.4,1.4,8.7,9.0
7,GATEWAY S,DON MILLS,0.6,1.4,8.8,10.0
8,GATEWAY S,DON MILLS,0.5,0.9,9.1,11.0
9,GATEWAY S,DON MILLS,0.7,2.3,11.4,12.0
